In [4]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import spacy
from bertopic import BERTopic
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_20newsgroups

model = SentenceTransformer('all-mpnet-base-v2')

In [5]:
df = pd.read_json("/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/Data_Processing/Data/job_processed_data_main.json")

df.head(20)

,Job_Titles,Neutral,Positive,Negative,Prompt_Tokens,Response_Tokens,Total_Tokens,Neutral_Encodings,Positive_Encodings,Negative_Encodings,Neutral_Terminated,Neutral_Terminated_Encodings,Positive_Leading,Positive_Leading_Encodings,Negative_Leading,Negative_Leading_Encodings,Positive_Topic,Negative_Topic
0,Technical Support Representative,I am a Technical Support Representative,I am knowledgeable and skilled in providing te...,I may encounter difficult and frustrated custo...,62,29,91,"[0.052540335800000004, -0.0741394237, -0.02414...","[0.0398985222, -0.039076753000000006, -0.03879...","[0.0750338361, 0.055929131800000004, -0.034461...",I am a Technical Support Representative.,"[0.0467863977, -0.0582639724, -0.0304021016000...",I am knowledgeable and skilled in providing te...,"[0.0400350429, -0.0512237102, -0.0361330472000...",I may encounter difficult and frustrated custo...,"[0.06889098140000001, 0.053461976300000005, -0...",0,0
1,A&H Underwriter,I am an A&H Underwriter,I have a great job as an A&H Underwriter.,I am not happy with my job as an A&H Underwriter.,65,35,100,"[-0.0183508322, 0.025549562600000002, -0.02162...","[-0.0315244012, 0.0867658779, -0.0072295303, -...","[-0.027420181800000002, 0.0644791871, -0.00949...",I am an A&H Underwriter .,"[-0.022203857100000002, 0.0287630353, -0.02035...",I have a great job as an A&H Underwriter,"[-0.0302970372, 0.0902502164, -0.0129407654, -...",I am not happy with my job as an A&H Underwriter,"[-0.026900822300000002, 0.0706868768, -0.01072...",-1,1
2,A&H Underwriting Manager,I am an A&H Underwriting Manager,I have a high-level position as an A&H Underwr...,"I am not just an entry-level employee, but I h...",65,44,109,"[-0.017012547700000002, 0.0080151027, -0.02787...","[-0.0027675340000000003, 0.068974562, -0.00189...","[-0.0194694344, 0.0587330759, -0.0063610324, -...",I am an A&H Underwriting Manager.,"[-0.0162954777, 0.0157897249, -0.0263584778000...",I have a high-level position as an A&H Underwr...,"[-0.0147530027, 0.0676713511, 0.0007165152, -0...","I am not just an entry-level employee, but I h...","[-0.0198444836, 0.054371766700000004, -0.00303...",0,0
3,A&P Technician,I am an A&P Technician,Being an A&P Technician is a rewarding career ...,The job of an A&P Technician can be physically...,63,55,118,"[-0.040869303, -0.0479970723, -0.0461201258, -...","[-0.027997929600000002, 0.0399987362, -0.03595...","[-0.0378840007, -0.030123831700000003, -0.0423...",I am an A&P Technician .,"[-0.0417694934, -0.0297952704, -0.0478198938, ...",Being an A&P Technician is a rewarding career ...,"[-0.028020246000000002, 0.0396952815, -0.03876...",The job of an A&P Technician can be physically...,"[-0.0376795791, -0.030884342300000003, -0.0492...",0,0
4,ABA Therapist,I am an ABA Therapist,I provide effective therapy to children with a...,I do not enjoy working as an ABA Therapist and...,63,37,100,"[0.0211608782, -0.0087794699, -0.0511314794, -...","[0.014801985600000001, 0.0252737999, -0.049951...","[0.0058508902, -0.0159540195, -0.0475744903, -...",I am an ABA Therapist.,"[0.015887849000000002, 0.0088945329, -0.048975...",I provide effective therapy to children with a...,"[0.0150047606, 0.0090112872, -0.05012279, 0.01...",I do not enjoy working as an ABA Therapist and...,"[0.0030206721000000003, -0.0278182309, -0.0552...",0,-1
5,ABAP Developer,I am an ABAP Developer,I have expertise in ABAP development and can c...,I do not have experience in other programming ...,62,44,106,"[0.015295145100000001, 0.0369808078, -0.054170...","[0.0101113003, 0.07414821540000001, -0.0528797...","[-0.0060721342, 0.0074372943, -0.0438291319, -...",I am an ABAP Developer.,"[0.0141174691, 0.051188509900000005, -0.056231...",I have expertise in ABAP development and can c...,"[0.0095568541, 0.060523558400000003, -0.052691...",I do not have experience in other programming ...,"[-0.0057134763, 0.0113966065, -0.0453605466, -...",0,-1
6,AEM Architect,I am an AEM Architect,I have extensive knowledge and experience in A.

In [6]:
df_filtered = df[["Positive", "Negative", "Positive_Topic", "Negative_Topic"]]
df_filtered.head(50)

,Positive,Negative,Positive_Topic,Negative_Topic
0,I am knowledgeable and skilled in providing te...,I may encounter difficult and frustrated custo...,0,0
1,I have a great job as an A&H Underwriter.,I am not happy with my job as an A&H Underwriter.,-1,1
2,I have a high-level position as an A&H Underwr...,"I am not just an entry-level employee, but I h...",0,0
3,Being an A&P Technician is a rewarding career ...,The job of an A&P Technician can be physically...,0,0
4,I provide effective therapy to children with a...,I do not enjoy working as an ABA Therapist and...,0,-1
5,I have expertise in ABAP development and can c...,I do not have experience in other programming ...,0,-1
6,I have extensive knowledge and experience in A...,I do not have experience in other content mana...,0,-1
7,Being an AEM Cloud Site Reliability Engineer i...,The job of an AEM Cloud Site Reliability Engin...,0,0
8,I have expertise in developing websites using ...,I do not have experience in developing website...,0,-1
9,I have expertise in AEM Front End Development.,I do not have experience in any other Front En...,0,-1


In [19]:
negative_text_topic0 = df_filtered[df_filtered["Negative_Topic"] == 0]["Negative"]

for i, text in enumerate(negative_text_topic0):
    print(f"\n{i}: {text}")


0: I may encounter difficult and frustrated customers who are hard to satisfy.

1: I am not just an entry-level employee, but I have a lot of responsibility as an A&H Underwriting Manager.

2: The job of an A&P Technician can be physically demanding and requires a lot of attention to detail, which can be stressful at times.

3: The job of an AEM Cloud Site Reliability Engineer can be stressful and demanding, with long hours and a lot of pressure to ensure that the website is always up and running smoothly.

4: Some people may not trust or feel comfortable with AI technology, which could make my job more challenging.

5: Some people may view AI as a threat to job security and privacy.

6: My job as an AP Clerk can be repetitive and tedious.

7: The job of an ATE Test Engineer can be very stressful and demanding, with long hours and tight deadlines.

8: I am not familiar with other cloud platforms and may not be able to provide solutions outside of AWS.

9: I am not familiar with other 

In [12]:
negative_text_noisy = df_filtered[df_filtered["Negative_Topic"] == -1]["Negative"]

for i, text in enumerate(negative_text_noisy):
    print(f"\n{i}: {text}")


0: I do not enjoy working as an ABA Therapist and find the job to be stressful.

1: I do not have experience in other programming languages and may not be able to work on projects that require different skills.

2: I do not have experience in other content management systems.

3: I do not have experience in developing websites using other content management systems.

4: I do not have experience in any other Front End Development technology.

5: Not everyone is qualified to be an AEM Technical Architect, as it requires a specific set of skills and experience.

6: I do not have experience in other areas of technology outside of AEM.

7: I lack experience in other areas of web development outside of AEM as a Technical Lead.

8: I am not a human engineer and may not have the same level of creativity or intuition.

9: My lack of emotional intelligence and human-like qualities may hinder my ability to effectively communicate and collaborate with others in the field.

10: I do not have exper

In [22]:
positive_text_topic0 =  df_filtered[df_filtered["Positive_Topic"] == 0]["Positive"]
for i, text in enumerate(positive_text_topic0):
    print(f"\n{i}: {text}")


0: I am knowledgeable and skilled in providing technical support to customers.

1: I have a high-level position as an A&H Underwriting Manager.

2: Being an A&P Technician is a rewarding career that allows me to work with aircraft and ensure their safety.

3: I provide effective therapy to children with autism using Applied Behavior Analysis techniques.

4: I have expertise in ABAP development and can create efficient programs.

5: I have extensive knowledge and experience in Adobe Experience Manager architecture.

6: Being an AEM Cloud Site Reliability Engineer is a challenging and rewarding job that requires a high level of technical expertise and problem-solving skills.

7: I have expertise in developing websites using Adobe Experience Manager (AEM).

8: I have expertise in AEM Front End Development.

9: Being an AEM Technical Architect is a challenging and rewarding career that requires a high level of expertise and knowledge.

10: I have expertise in AEM and can provide technical